In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-9d8cf0df-03dd-6020-4269-62b9a66c4d5b)


In [2]:
dataset_path = "/home/jupyter/cognitive/tiny-imagenet-200"
dataset_training_path = "/home/jupyter/cognitive/tiny-imagenet-200/train"
dataset_val_path = "/home/jupyter/cognitive/tiny-imagenet-200/val"
dataset_stats = "/home/jupyter/cognitive/normalise_params.txt"
dataset_id_idx = "/home/jupyter/cognitive/tiny-imagenet-200/wnids.txt"

In [3]:
!pip install efficientnet-pytorch

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

# import sys

# sys.path.append("/content/drive/MyDrive/cognitive_robotics/")

In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import json
import random

import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on: {device}")

Running on: cuda


In [6]:
from TinyImageNetDataset import TinyImageNetDataset
from model import get_efficientnet
from utils import save_checkpoint, log
from data_transformations import get_data_transformations
from train_val import train_val

<function get_id_to_idx at 0x7f8b477c14d0>


In [7]:
NUM_EXPERIMENT_REPETITIONS = 3

In [8]:
def experiment(model_type, epoch_count, learning_rate, dropout_rate, pretrained,
               warm_up, crop, horizontal_flip_rate, mixup, repetitions =  NUM_EXPERIMENT_REPETITIONS
              ):
  """
    model_type: efficientnet size used
    epoch_count: number of epochs spent training 
    learning rate: learning rate in gradient descent
    dropout_rate: dropout rate of the model
    pretrained: is the model pretrained on ImageNet
    warm_up: number of iterations when warm up will be applied
    crop: should perform central crop
    flip: rate of flipping images horizontally
    mixup: how often should mixup be applied
  """
  means = np.zeros((repetitions, 2))
  for repetition in range(repetitions):
    torch.manual_seed(repetition)
    random.seed(repetition)
    model = get_efficientnet(model_type, pretrained, dropout_rate)
    with open(dataset_stats) as fp:
      stats = json.load(fp=fp)

    dataset_train = TinyImageNetDataset(
      dataset_training_path, 
      dataset_id_idx, 
      get_data_transformations(stats),
      True
    )
    print(dataset_train.__len__())
    dataset_val = TinyImageNetDataset(
      dataset_val_path, 
      dataset_id_idx,
      get_data_transformations(stats),
      False
    )
    print(dataset_val.__len__())

    has_augment = True if crop or horizontal_flip_rate > 0 else False

    dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=40, shuffle=True, pin_memory=True, num_workers=6)
    dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=48, shuffle=False, pin_memory=False, num_workers=4)
    model.to(device)
    print("Starting training")
    means[repetition] = train_val(model, model_type, dataloader_train, dataloader_val, learning_rate, epoch_count, 
              dropout_rate, pretrained, warm_up, has_augment, mixup)
  print("Mean: ", np.mean(means, 1))
  print("Std: ", np.std(means, axis=1))


In [9]:
# import pandas as pd
# import os

# val_data = pd.read_csv(f'{dataset_val_path}/val_annotations.txt', 
#                        sep='\t', 
#                        header=None, 
#                        names=['File', 'Class', 'X', 'Y', 'H', 'W'])

# val_data.head()

# # Create separate validation subfolders for the validation images based on
# # their labels indicated in the val_annotations txt file
# val_img_dir = os.path.join(dataset_val_path, 'images')

# # Open and read val annotations text file
# fp = open(os.path.join(dataset_val_path, 'val_annotations.txt'), 'r')
# data = fp.readlines()

# # Create dictionary to store img filename (word 0) and corresponding
# # label (word 1) for every line in the txt file (as key value pair)
# val_img_dict = {}
# for line in data:
#     words = line.split('\t')
#     val_img_dict[words[0]] = words[1]
# fp.close()

# # Display first 10 entries of resulting val_img_dict dictionary
# {k: val_img_dict[k] for k in list(val_img_dict)[:10]}

# # and move images into the respective folders
# for img, folder in val_img_dict.items():
#     newpath = (os.path.join(val_img_dir, folder))
#     if not os.path.exists(newpath):
#         os.makedirs(newpath)
#     if os.path.exists(os.path.join(val_img_dir, img)):
#         os.rename(os.path.join(val_img_dir, img), os.path.join(newpath, img))

Determine optimal learning rate for EfficientNet-B1

In [ ]:
# experiment("efficientnet-b1", 30, 0.001, 0.4, True, 0, True, 0.5, 0, 1)
experiment("efficientnet-b1", 10, 0.1, 0.4, True, 0, True, 0.5, 0, 1)
experiment("efficientnet-b1", 10, 0.01, 0.4, True, 0, True, 0.5, 0, 1)


Loaded pretrained weights for efficientnet-b1
/home/jupyter/cognitive/tiny-imagenet-200/train
200
100000
/home/jupyter/cognitive/tiny-imagenet-200/val/images
200
10000


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Starting training
Running on cuda
Epoch: 0


Iterations, training: 100%|██████████| 2778/2778 [1:04:07<00:00,  1.38s/it]



Total loss: 16540.20763349533


...

Iterations, training: 100%|██████████| 2778/2778 [1:04:14<00:00,  1.39s/it]



Total loss: 8216.883172512054


Iterations, validation: 100%|██████████| 209/209 [01:47<00:00,  1.95it/s]


Loss: 661.8270815014839
Accuracy top 1: 0.27
Accuracy top 5: 0.54
Epoch: 8


Iterations, training:  62%|██████▏   | 1734/2778 [40:05<24:09,  1.39s/it] 

In [ ]:
experiment("efficientnet-b0", 10, 0.1, 0.4, True, 0, True, 0.5, 0, 1)
experiment("efficientnet-b0", 10, 0.01, 0.4, True, 0, True, 0.5, 0, 1)


Loaded pretrained weights for efficientnet-b0
/home/jupyter/cognitive/tiny-imagenet-200/train
200
100000
/home/jupyter/cognitive/tiny-imagenet-200/val/images
200
10000


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Starting training
Running on cuda
Epoch: 0


Iterations, training: 100%|██████████| 2500/2500 [42:36<00:00,  1.02s/it]



Total loss: 13763.334493160248


...

Iterations, training: 100%|██████████| 2500/2500 [42:42<00:00,  1.02s/it]



Total loss: 6416.954568147659


Iterations, validation: 100%|██████████| 209/209 [01:12<00:00,  2.88it/s]


Loss: 545.6022572517395
Accuracy top 1: 0.38
Accuracy top 5: 0.66
Epoch: 9


Iterations, training:  69%|██████▉   | 1732/2500 [29:36<13:07,  1.03s/it]

In [13]:
experiment("efficientnet-b1", 2, 0.01, 0.4, True, 0, True, 0.5, 0, 1)


Loaded pretrained weights for efficientnet-b1


AttributeError: 'function' object has no attribute 'copy'